In [1]:
import gc
import traceback
import os
from tqdm import tqdm
import pandas as pd
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
from policyengine_us_data import Pooled_3_Year_CPS_2023

In [2]:
def define_reforms():
    reforms = {
        "reform": Reform.from_dict({
            "gov.contrib.ubi_center.basic_income.amount.person.flat": {
                "2025-01-01.2025-12-31": 1160,
                "2026-01-01.2026-12-31": 1605,
                "2027-01-01.2027-12-31": 1686
            }
        }, country_id="us"),

        "reform_taxable": Reform.from_dict({
            "gov.contrib.states.or.rebate.state_tax_exempt": {
                "2024-01-01.2100-12-31": True
            },
            "gov.contrib.ubi_center.basic_income.amount.person.flat": {
                "2025-01-01.2025-12-31": 1160,
                "2026-01-01.2026-12-31": 1605,
                "2027-01-01.2027-12-31": 1686
            },
            "gov.contrib.ubi_center.basic_income.taxable": {
                "2024-01-01.2100-12-31": True
            }
        }, country_id="us"),

        "reform_flat_tax": Reform.from_dict({
            "gov.contrib.ubi_center.basic_income.amount.person.flat": {
                "2025-01-01.2025-12-31": 1160,
                "2026-01-01.2026-12-31": 1605,
                "2027-01-01.2027-12-31": 1686
            },
            "gov.contrib.ubi_center.flat_tax.rate.agi": {
                "2025-01-01.2025-12-31": 0.0187,
                "2026-01-01.2026-12-31": 0.025,
                "2027-01-01.2028-12-31": 0.0254
            }
        }, country_id="us"),

        "reform_taxable_flat_tax": Reform.from_dict({
            "gov.contrib.states.or.rebate.state_tax_exempt": {
                "2024-01-01.2100-12-31": True
            },
            "gov.contrib.ubi_center.basic_income.amount.person.flat": {
                "2025-01-01.2025-12-31": 1160,
                "2026-01-01.2026-12-31": 1605,
                "2027-01-01.2027-12-31": 1686
            },
            "gov.contrib.ubi_center.basic_income.taxable": {
                "2024-01-01.2100-12-31": True
            },
            "gov.contrib.ubi_center.flat_tax.rate.agi": {
                "2025-01-01.2025-12-31": 0.0187,
                "2026-01-01.2026-12-31": 0.025,
                "2027-01-01.2028-12-31": 0.0254
            }
        }, country_id="us")
    }
    return reforms

In [3]:
def initialize_microsimulations(reforms):
    baseline = Microsimulation(dataset=Pooled_3_Year_CPS_2023)
    reformed_sims = {name: Microsimulation(reform=reform, dataset=Pooled_3_Year_CPS_2023) 
                     for name, reform in reforms.items()}
    return baseline, reformed_sims

In [4]:
def calculate_poverty_impact_for_age_group(baseline, reformed, year, min_age, max_age, label):
    try:
        state_codes = baseline.calc("state_code", map_to="person", period=year)
        age = baseline.calc("age", map_to="person", period=year)
        baseline_poverty = baseline.calc("in_poverty", map_to="person", period=year)
        reform_poverty = reformed.calc("in_poverty", map_to="person", period=year)

        mask = (state_codes == "OR") & (age >= min_age) & (age < max_age) if label != "Overall" else (state_codes == "OR")
        baseline_poverty_group = baseline_poverty[mask].mean()
        reform_poverty_group = reform_poverty[mask].mean()
        relative_poverty_reduction = (reform_poverty_group - baseline_poverty_group) / baseline_poverty_group

        return relative_poverty_reduction
    except Exception as e:
        print(f"Error calculating poverty impact for age group {label} in year {year}: {str(e)}")
        traceback.print_exc()
        return None

In [5]:
def process_age_group(year, baseline, reformed, reform_name, min_age, max_age, label):
    try:
        relative_poverty_reduction = calculate_poverty_impact_for_age_group(baseline, reformed, year, min_age, max_age, label)
        if relative_poverty_reduction is not None:
            result = {
                "year": year,
                "age_group": label,
                "reform": reform_name,
                "relative_poverty_reduction": relative_poverty_reduction
            }
            checkpoint_path = f"or_rebate_{year}_{reform_name}_{label.replace('-', '_')}.csv"
            pd.DataFrame([result]).to_csv(checkpoint_path, index=False)
            print(f"Results for year {year}, reform {reform_name}, age group {label} saved to {checkpoint_path}")
            return result
        return None
    except Exception as e:
        print(f"Error processing age group {label} for year {year} and reform {reform_name}: {str(e)}")
        traceback.print_exc()
        return None


In [6]:
def get_processed_combinations(start_year, end_year, reforms):
    processed = set()
    for year in range(start_year, end_year):
        for reform_name in reforms.keys():
            for label in ["0-17", "18-64", "65+", "Overall"]:
                file_name = f"or_rebate_{year}_{reform_name}_{label.replace('-', '_')}.csv"
                if os.path.exists(file_name):
                    processed.add((year, reform_name, label))
    return processed

In [7]:
if __name__ == "__main__":
    try:
        reforms = define_reforms()
        baseline, reformed_sims = initialize_microsimulations(reforms)
        print("Microsimulations initialized.")

        start_year = 2025
        end_year = 2028
        age_groups = [
            (0, 18, "0-17"),
            (18, 65, "18-64"),
            (65, 200, "65+"),
            (0, 200, "Overall")
        ]

        processed_combinations = get_processed_combinations(start_year, end_year, reforms)
        
        all_results = []
        for year in tqdm(range(start_year, end_year), desc="Processing years"):
            for reform_name, reformed in reformed_sims.items():
                for min_age, max_age, label in age_groups:
                    if (year, reform_name, label) not in processed_combinations:
                        result = process_age_group(year, baseline, reformed, reform_name, min_age, max_age, label)
                        if result:
                            all_results.append(result)
                    else:
                        print(f"Skipping already processed: Year {year}, Reform {reform_name}, Age group {label}")
                        file_name = f"or_rebate_{year}_{reform_name}_{label.replace('-', '_')}.csv"
                        all_results.append(pd.read_csv(file_name).iloc[0].to_dict())
            
            # Clear some memory after each year
            gc.collect()

        final_results = pd.DataFrame(all_results)
        final_csv_path = "or_rebate_all_results.csv"
        final_results.to_csv(final_csv_path, index=False)
        print(f"All results saved to {final_csv_path}")
    except Exception as e:
        print(f"An error occurred in the main execution: {str(e)}")
        traceback.print_exc()

Microsimulations initialized.


Processing years:   0%|          | 0/3 [00:50<?, ?it/s]


KeyboardInterrupt: 